# Google Colab でオープンソース大規模LLMをファインチューニング

このノートブックでは、Hugging Face Hub からベースモデルを取得し、LoRA/QLoRA を使用してファインチューニングを行います。

⚠️ **注意事項**:
- Google Colab Pro/Pro+ での実行を推奨します
- GPU (T4, L4, A100) が必要です
- 学習には数時間かかる場合があります
- セッション切断に備えて、こまめに保存しましょう

## 設定パラメータ

以下のパラメータを必要に応じて変更してください。

In [ ]:
# プロジェクト設定
PROJECT_NAME = "gpt-oss-lora-demo"
BASE_SAVE_DIR = "/content/drive/MyDrive/llm-finetune"

# モデル設定
BASE_MODEL_ID = "Qwen/Qwen2-7B"  # 例: 実運用向け 7B モデル
# BASE_MODEL_ID = "gpt-oss/gpt-oss-120B"  # 例: 超大規模モデル（テンプレ）
USE_4BIT = True  # QLoRA用の4bit量子化を使用

# データセット設定
DATASET_ID = "iac/ja-wiki-2023"  # 日本語データセットの例
DATASET_SPLIT = "train"
TEXT_COLUMN = "text"
MAX_SEQ_LENGTH = 1024
PACKING = True  # 複数文書を詰めて固定長シーケンスにする

# LoRA / トレーニング設定
LORA_R = 64
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = ["q_proj", "v_proj"]  # モデルに応じて調整

BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 16
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 2
WARMUP_STEPS = 100
LOGGING_STEPS = 10
SAVE_STEPS = 500
MAX_STEPS = -1  # 制限しない場合は -1

# 出力ディレクトリ
OUTPUT_DIR = f"{BASE_SAVE_DIR}/{PROJECT_NAME}"
FINAL_DIR = f"{OUTPUT_DIR}/final"

print(f"プロジェクト名: {PROJECT_NAME}")
print(f"ベースモデル: {BASE_MODEL_ID}")
print(f"データセット: {DATASET_ID}")
print(f"保存先: {OUTPUT_DIR}")

## 1. 環境準備 & Google Drive マウント

### GPU情報の確認

In [ ]:
!nvidia-smi

### 依存ライブラリのインストール

In [ ]:
!pip install -U "transformers[torch]" datasets accelerate peft bitsandbytes huggingface_hub sentencepiece trl

### Google Drive のマウント

In [ ]:
from google.colab import drive
import os

# Google Drive をマウント
drive.mount('/content/drive')

# 保存先ディレクトリの作成
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(FINAL_DIR, exist_ok=True)

print(f"Google Drive をマウントしました")
print(f"保存先ディレクトリ: {OUTPUT_DIR}")

## 2. Hugging Face 認証 & ベースモデル設定

### Hugging Face トークンの設定

In [ ]:
import getpass
from huggingface_hub import login

# Hugging Face トークンを安全に入力
HF_TOKEN = getpass.getpass("Hugging Face Token を入力してください: ")

# Hugging Face にログイン
login(token=HF_TOKEN)

print("Hugging Face にログインしました")

### トークナイザの読み込み

In [ ]:
from transformers import AutoTokenizer

# トークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_ID,
    trust_remote_code=True
)

# パディングトークンの設定（必要な場合）
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"トークナイザを読み込みました: {BASE_MODEL_ID}")
print(f"語彙サイズ: {len(tokenizer)}")

## 3. データセット取得 & 前処理

### データセットの読み込み

In [ ]:
from datasets import load_dataset

# データセットの読み込み
print(f"データセットを読み込んでいます: {DATASET_ID}")
dataset = load_dataset(DATASET_ID, split=DATASET_SPLIT)

# サンプル数を制限する場合（メモリ節約のため）
# dataset = dataset.select(range(min(10000, len(dataset))))

print(f"データセットサイズ: {len(dataset)}")
print(f"サンプル例: {dataset[0]}")

### データセットの前処理（トークナイズ）

In [ ]:
def tokenize_function(examples):
    """テキストをトークナイズする関数"""
    return tokenizer(
        examples[TEXT_COLUMN],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding=False,
        return_special_tokens_mask=True
    )

# データセット全体をトークナイズ
print("データセットをトークナイズしています...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
    desc="トークナイズ中"
)

print(f"トークナイズ完了: {len(tokenized_dataset)} サンプル")

## 4. トレーニング設定（LoRA / QLoRA）

### 4bit 量子化設定（QLoRA）

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# 4bit 量子化の設定
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    print("4bit 量子化設定を使用します（QLoRA）")
else:
    bnb_config = None
    print("量子化なしでモデルを読み込みます")

### ベースモデルの読み込み

In [ ]:
# ベースモデルの読み込み
print(f"ベースモデルを読み込んでいます: {BASE_MODEL_ID}")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if not USE_4BIT else None
)

# グラディエント計算を有効化（量子化モデルの場合）
if USE_4BIT:
    model.config.use_cache = False
    model = model.to(torch.bfloat16)

print("ベースモデルを読み込みました")
print(f"モデルパラメータ数: {model.num_parameters():,}")

### LoRA 設定の適用

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 量子化モデルの準備
if USE_4BIT:
    model = prepare_model_for_kbit_training(model)

# LoRA 設定
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=LORA_TARGET_MODULES,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA アダプタを注入
model = get_peft_model(model, lora_config)

# 学習可能パラメータの表示
model.print_trainable_parameters()

print("LoRA アダプタを適用しました")

## 5. 学習実行

### トレーニング引数の設定

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# トレーニング引数
training_args = TrainingArguments(
    output_dir=f"{OUTPUT_DIR}/checkpoints",
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    max_steps=MAX_STEPS,
    warmup_steps=WARMUP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    bf16=True,
    evaluation_strategy="no",
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit" if USE_4BIT else "adamw_torch",
    gradient_checkpointing=True,
    report_to="none",  # WandB等を使う場合は変更
    save_safetensors=True,
    push_to_hub=False,
)

# データコレータ（MLMタスク用）
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM のため False
)

print("トレーニング引数を設定しました")

### Trainer の初期化と学習実行

In [ ]:
# Trainer の初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Trainer を初期化しました")
print("学習を開始します...")

# 学習実行
train_result = trainer.train()

print("学習が完了しました！")
print(f"トレーニング損失: {train_result.training_loss}")

## 6. 学習済みモデルの Google Drive への保存

In [ ]:
# LoRA アダプタとトークナイザを保存
print(f"モデルを保存しています: {FINAL_DIR}")

model.save_pretrained(FINAL_DIR)
tokenizer.save_pretrained(FINAL_DIR)

print("保存完了！")
print(f"保存先: {FINAL_DIR}")

# 保存されたファイルの確認
!ls -lh {FINAL_DIR}

## 7. 学習済みモデルを用いた推論デモ

### 学習済みモデルの読み込み

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

print("学習済みモデルを読み込んでいます...")

# ベースモデルの読み込み
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# トークナイザの読み込み
inference_tokenizer = AutoTokenizer.from_pretrained(FINAL_DIR)

# LoRA アダプタの読み込み
inference_model = PeftModel.from_pretrained(base_model, FINAL_DIR)
inference_model.eval()

print("モデルの読み込み完了！")

### 推論関数の定義

In [ ]:
def generate_text(prompt, max_new_tokens=128, temperature=0.7, top_p=0.95):
    """
    学習済みモデルを使ってテキストを生成する関数
    
    Args:
        prompt: 入力プロンプト
        max_new_tokens: 生成する最大トークン数
        temperature: サンプリング温度
        top_p: nucleus サンプリングのパラメータ
    
    Returns:
        生成されたテキスト
    """
    inputs = inference_tokenizer(prompt, return_tensors="pt").to(inference_model.device)
    
    with torch.no_grad():
        outputs = inference_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            pad_token_id=inference_tokenizer.pad_token_id,
            eos_token_id=inference_tokenizer.eos_token_id
        )
    
    return inference_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("推論関数を定義しました")

### テスト推論

In [ ]:
# テストプロンプト
test_prompts = [
    "人工知能とは",
    "機械学習の応用例として",
    "深層学習の特徴は"
]

print("テスト推論を実行します:\n")

for prompt in test_prompts:
    print(f"プロンプト: {prompt}")
    result = generate_text(prompt, max_new_tokens=100)
    print(f"生成結果: {result}")
    print("-" * 80 + "\n")

### 対話的推論（任意のプロンプトで試す）

In [ ]:
# 任意のプロンプトで推論を実行
custom_prompt = input("プロンプトを入力してください: ")

result = generate_text(custom_prompt, max_new_tokens=200)
print(f"\n生成結果:\n{result}")

## 8. オプション: Hugging Face Hub への push

### Hub へのアップロード（オプション）

⚠️ **注意**: 
- Hugging Face トークンに `write` 権限が必要です
- ベースモデルとデータセットのライセンスを確認してください
- 公開する場合は適切なモデルカードを作成してください

In [ ]:
# Hugging Face Hub にアップロードする場合は以下のコメントを外して実行

# HUB_MODEL_NAME = "your-username/your-model-name"  # 変更してください

# print(f"Hugging Face Hub にアップロードしています: {HUB_MODEL_NAME}")

# # モデルとトークナイザをアップロード
# model.push_to_hub(HUB_MODEL_NAME, use_auth_token=HF_TOKEN)
# tokenizer.push_to_hub(HUB_MODEL_NAME, use_auth_token=HF_TOKEN)

# print("アップロード完了！")
# print(f"モデルURL: https://huggingface.co/{HUB_MODEL_NAME}")

## まとめ

このノートブックでは以下を実行しました：

1. Google Colab 環境の準備と Google Drive のマウント
2. Hugging Face からベースモデルとデータセットの取得
3. LoRA/QLoRA を使用したパラメータ効率的なファインチューニング
4. 学習済みモデルの Google Drive への保存
5. 学習済みモデルを使った推論デモ

学習済みモデルは `{OUTPUT_DIR}` に保存されています。

### 次のステップ

- 異なるデータセットで学習してみる
- ハイパーパラメータを調整して性能を改善する
- WandB や TensorBoard を使って学習を可視化する
- 学習済みモデルを本番環境にデプロイする